# Ipywidget을 활용한 interactive EDA
Growth Hackers 2019-2 시각화 스터디 후반부
- 참여자: 이경 찬영 현영
- 기간: 2019/10/01 ~ 10/10
- 참고: https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6

In [1]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import seaborn as sns
import pandas as pd

In [2]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


## Interactive Data Selection
### `interact`
1. 함수로 활용  
: function을 arguement로 받으며, 해당 function이 필요로하는 파라미터를 추가적으로 받은 뒤 function을 __호출__
2. 데코레이터로 활용  
: function 정의하고 interact 함수 또 호출 하는 게 귀찮다! => 데코레이터로 한 번에  
  
### basics
- interact를 사용할 때는 모든 파라미터의 default 값을 지정해줘야 한다(non-default argument follows default argument)
- default 값을 무엇으로 주는냐에 따라 slider의 범위가 바뀐다. slider 범위는 column 상관 없이 고정 --> default에 범위가 가장 포괄적인 column을 넣어줘야 다른 column에도 쓸 수 있다
- interact하는 변수들은 기본적으로 categorical이면 text box, numeric이면 slider

In [3]:
@interact
def show_articles_more_than(column='total_bill', x=20):
    return df.loc[df[column] > x]

interactive(children=(Text(value='total_bill', description='column'), IntSlider(value=20, description='x', max…

### sliders with specification of arguments
- categorical variable에 텍스트박스 대신 dropdown 메뉴: `paramenter_name = [ 변수명1, 변수명2, ... ]`  
- 슬라이더 상세설정: `x = (시작 값, 끝 값, 간격)`

In [4]:
@interact
def show_items_more_than(column=['total_bill', 'tip', 'size'],
                            x=(0, 50, 1)):
    return df.loc[df[column] > x]

interactive(children=(Dropdown(description='column', options=('total_bill', 'tip', 'size'), value='total_bill'…

## Interactive Plotting
selecting과 동일하게 `interact` 데코레이터를 쓴 후 함수 정의만 해줘도 그래프가 바로 찍힌다

In [5]:
# numeric column만 선택하기
df.select_dtypes('number').columns

Index(['total_bill', 'tip', 'size'], dtype='object')

In [6]:
# 하이퍼 파라미터에 칼럼 명의 리스트를 전달하여 dropdown 메뉴로 나타나도록
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns),
                 y=list(df.select_dtypes('number').columns),
                 hue = list(df.columns),
                 size = list(df.columns)):

    sns.relplot(kind='scatter', x = x, y = y, hue = hue, size = size, height = 6, aspect = 1.2, data = df)

interactive(children=(Dropdown(description='x', options=('total_bill', 'tip', 'size'), value='total_bill'), Dr…

In [7]:
# numeric이 아닌 = categorical 칼럼 선택하기
df.select_dtypes(exclude=["number"]).columns

Index(['sex', 'smoker', 'day', 'time'], dtype='object')

In [9]:
# use seaborn
# add categorical hue
@interact
def scatter_plot(x=list(df.select_dtypes('number').columns),
                 y=list(df.select_dtypes('number').columns),
                 cat=list(df.select_dtypes(exclude=["number"]).columns)):
    
    sns.relplot(x=x, y=y, hue=cat, palette="ch:r=-.5,l=.75", data=df)

interactive(children=(Dropdown(description='x', options=('total_bill', 'tip', 'size'), value='total_bill'), Dr…

## Datetime Picker

### 기존의 데이터에 datetime column 추가
- datetime picker는 datetime 형태의 칼럼에 `widgets.DatePicker(value=pd.to_datetime('2019-01-01'))` 형태로 날짜 선택기를 넣어준다
- 원 데이터 프레임의 column도 datetime object여야한다. 아니었다면 `pd.to_datetime`으로 바꿔주기!
```python
pd.to_datetime('13000101', format='%Y%m%d', errors='ignore')
# => datetime.datetime(1300, 1, 1, 0, 0)
```

In [12]:
# function to generate random datetime objects
import random
import time

def str_time_prop(start, end, format, prop):
    """Get a time at a proportion of a range of two formatted times.

    start and end should be strings specifying times formated in the
    given format (strftime-style), giving an interval [start, end].
    prop specifies how a proportion of the interval to be taken after
    start.  The returned time will be in the specified format.
    """

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))


def random_date(start, end, prop):
    return str_time_prop(start, end, '%Y-%m-%d', prop)

In [13]:
# test random_date function
random_date("2019-01-01", "2019-06-30", random.random())

'2019-01-23'

In [15]:
# make a new column 
date = [random_date("2019-01-01", "2019-06-30", random.random()) for i in range(len(df))]
df['date'] = pd.to_datetime(date)
df.head()

,total_bill,tip,sex,smoker,day,time,size,date
0,16.99,1.01,Female,No,Sun,Dinner,2,2019-02-01
1,10.34,1.66,Male,No,Sun,Dinner,3,2019-04-29
2,21.01,3.50,Male,No,Sun,Dinner,3,2019-02-14
3,23.68,3.31,Male,No,Sun,Dinner,2,2019-01-12
4,24.59,3.61,Female,No,Sun,Dinner,4,2019-04-17


### 위젯 사용하기

In [35]:
widgets.DatePicker(value=pd.to_datetime('2019-01-01'))

ipywidgets.widgets.widget_date.DatePicker

In [16]:
# 방법 1 - 함수 안에서 위젯쓰기
@interact
def print_datetime(start_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')),
        end_date=widgets.DatePicker(value=pd.to_datetime('2019-06-30'))):
    
    data = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    print(data.head())

interactive(children=(DatePicker(value=Timestamp('2019-01-01 00:00:00'), description='start_date'), DatePicker…

In [58]:
# 방법 2 - 따로 함수를 지정한 뒤, interact에 함수를 변수로 전달하기
# datetime 두 개를 파라미터로 받는 함수가 여러개 있으면 재사용
def print_datetime(start_date, end_date):
    data = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    print(data.head())

# _에 할당 왜 해줬는지? 할당 해도 안해도 결과가 뜨는 건 똑같다
_ = interact(print_datetime,
        start_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')),
        end_date=widgets.DatePicker(value=pd.to_datetime('2019-06-30')))

interactive(children=(DatePicker(value=Timestamp('2019-01-01 00:00:00'), description='start_date'), DatePicker…

### 실험

In [66]:
# function object 그 자체를 저장한 오브젝트. 그래프가 보여지고 말고는 이것을 return하는 것과 관계 없다
_

<function print_datetime at 0x7fcaf83980d0>


In [68]:
# 위의 interact를 함수를 받을 수 있는 함수로 변환
def datetime_interactor(f):
    # return을 해도 안해도 picker와 head를 보여주는 것 동일
    # return을 할 경우 <function .. 어쩌구 해서 interact object자체를 출력함
    return interact(f,
        start_date=widgets.DatePicker(value=pd.to_datetime('2019-01-01')),
        end_date=widgets.DatePicker(value=pd.to_datetime('2019-06-30')))

datetime_interactor(print_datetime)

interactive(children=(DatePicker(value=Timestamp('2019-01-01 00:00:00'), description='start_date'), DatePicker…

<function __main__.print_datetime(start_date, end_date)>

`interact`는 먼가 특성이 다른 듯 다른 변수랑...

## Hover Interaction

###### `pd.Timestamp`란  
https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.Timestamp.html
- python 내장 datetime과 비슷한데 TimestampIndex를 만들기 위해 <--먼말?

###### plotly  
https://plot.ly/python/hover-text-and-formatting/

In [17]:
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import seaborn as sns
import pandas as pd

import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

def plot_up_to(column, date):
    date = pd.Timestamp(date) 
    plot_df = df.loc[df['date'] <= date].copy() # 해당하는 날짜까지만 df 선택하기
    plot_df[column].cumsum().iplot(mode='markers+lines',
                                   xTitle='date for bills',
                                   yTitle=column,
                                  title=f'Cumulative {column.title()} Until {date.date()}') # 타이틀에 선택된 칼럼 및 날짜 string으로 넣기
    
_ = interact(plot_up_to, column=widgets.Dropdown(options=list(df.select_dtypes('number').columns)), 
             date = widgets.DatePicker(value=pd.to_datetime('2019-03-01')))

interactive(children=(Dropdown(description='column', options=('total_bill', 'tip', 'size'), value='total_bill'…